In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
np.random.seed(0)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [45]:
#a. Importar os arquivos “kaggle/input/titanic/test.csv” e “kaggle/input/titanic/train.csv”
test = pd.read_csv("/kaggle/input/titanic/test.csv")
train = pd.read_csv("/kaggle/input/titanic/train.csv")
    

In [46]:
#b. Gerar momentos estatísticos sobre o dataframe de treino, utilizando “.describe( )”
train.describe(include = "all")


In [42]:
#c. Identificar as Features do banco de dados que são categóricas (não numéricas), 
# e as que são numéricas. As numéricas, deve-se separá-las em
# discretas e contínuas
train.info()

In [49]:
#d. Descobrir quais as Features contém NaN, além da quantidade desse tipo de
# dado em cada Feature.
pd.isnull(train).sum()

In [69]:
#e. Deve-se utilizar apenas Features numéricas e retirar as categóricas. Portanto,
# faça dois novos dataframes, apenas com as features numéricas do banco de
# dados de treino e de teste.
train_num = train.loc[:, ["PassengerId","Survived","Age","SibSp","Parch","Fare","Pclass"]]
#retirando as linhas com NaN
train_num = train_num.dropna(axis = 0)
test_num = test.loc[:, ["PassengerId","Age","SibSp","Parch","Fare","Pclass"]]
#substituindo os valores Nan por 0
test_num = test_num.fillna(0)
print(train_num.shape)
print(test_num.shape)


In [84]:
#Para a Feature “Age” será necessário criar uma nova Feature que a separe em
# 8 grupos para que se possa utilizar “sns.barplot”.

#f. Criar uma lista com os nomes que serão dados aos grupos em que as
# idades serão repartidas (Exemplo: “(0-10)”, “(10-20)”, ...)
train_age = train_num.loc[:, ["Survived", "Age"]]

vetDivisao = np.arange(0,90,10)
divisao = ["0-10","10-20","20-30","30-40","40-50","50-60","60-70","70-80"]

#agrupando em faixa etária e criando nova coluna
train_age["AgeGroup"] = pd.cut(train_num["Age"], vetDivisao, labels = divisao)
train_age.head()

#plotando em gráfico as chances de sobrevivência
plt.title("Sobrevivência quanto à idade")
display(sns.barplot(x="AgeGroup",y="Survived", data=train_age))


In [81]:
plt.title("Sobrevivência quanto ao fato de ter parentes a bordo")
sns.barplot(x="SibSp",y="Survived", data=train_num)

In [79]:
plt.title("Sobrevivência quanto ao fato de não ter pais ou filhos a bordo")
sns.barplot(x="Parch",y="Survived", data=train_num)

In [80]:
plt.title("Sobrevivência quanto à classe da viagem")
sns.barplot(x="Pclass",y="Survived", data=train_num)

In [110]:
#Separação de banco de dados de treino e validação.

from sklearn.model_selection import train_test_split

#var contendo o banco de dados apenas das Features
X = train_num.drop(["PassengerId", "Survived"], axis = 1)
#var contendo a Target da predição
Y = train_num["Survived"]
#utilizando train test split para separar treino e validação (validação com 20% do tamanho do treino)
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size = 0.2, random_state = 0)

#Treinamento e validação do modelo.
#Classifier se diferencia do Regressor por gerar predições de números discrestos

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#Treinamento
randomforest = RandomForestClassifier()
randomforest.fit(train_X, train_Y)
preds = randomforest.predict(val_X)

#Validação
accurance = round(accuracy_score(preds,val_Y)*100,2)
print(accurance)

In [121]:
#Gerar predição, salvar a versão e submeter à competição

#Aplicando o modelo ao "Test"
index = test_num["PassengerId"]
prediction = randomforest.predict(test_num.drop("PassengerId", axis = 1))

#Arquivo com as predições
output = pd.DataFrame({"PassengerId": index, "Survived": prediction})
output.to_csv("submission.csv", index = False)


pd.read_csv("/kaggle/working/submission.csv")